In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

import re
import os
import io
import time
import random

import seaborn # Attention 시각화를 위해 필요!
from sklearn.model_selection import train_test_split
import glob

### Loading Data

In [3]:
path1 = os.getenv('HOME') + '/aiffel/Deep_Sum/102/20201021-20201031' # 자신에 맞는 숫자 집어넣기
path2 = os.getenv('HOME') + '/aiffel/Deep_Sum/102/20201011-20201020' # path
path3 = os.getenv('HOME') + '/aiffel/Deep_Sum/102/20201001-20201010' # path
filenames1 = glob.glob(path1 + "/*.csv") 
filenames2 = glob.glob(path2 + '/*.csv')
filenames3 = glob.glob(path3 + '/*.csv')

dfs = []
for filename in filenames1:
    dfs.append(pd.read_csv(filename))
    
#for filename in filenames2:
#    dfs.append(pd.read_csv(filename))

#for filename in filenames3:
#    dfs.append(pd.read_csv(filename))


data = pd.concat(dfs, ignore_index=True)
print(data.shape)

(83214, 7)


In [4]:
data = data.drop_duplicates(['contents'], ignore_index = True)

In [5]:
document = data['contents']
summary = data['title']
print(document.head())
print(summary.head())
print(len(document))

0    오늘(30일) 밤 8시 40분쯤, 경기도 화성시 우정읍 우정읍사무소 인근 왕복 2차...
1    기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...
2    [한국경제TV 조시형 기자]\n\n전북 군산 만경강에서 지난 26일 채취한 야생조류...
3    서울 송파구 서울동부지검. 2019.4.1/뉴스1 © News1 구윤성 기자 서울 ...
4    동영상 뉴스\n\n[앵커]핼러윈을 맞아 서울 이태원 등 번화가에는 젊은이들의 발길이...
Name: contents, dtype: object
0              화성시 도로서 경차-트럭-트랙터 부딪혀…“2명 부상”
1                             '홍대거리 방역조치 점검'
2    군산 만경강 야생조류 분변서 AI 항원 검출…"고병원성 여부 확인 중"
3           검찰, '소윤' 윤대진 친형 '변호사법 위반' 참고인 조사
4                  핼러윈 맞은 이태원 밤은...'기대 속 긴장'
Name: title, dtype: object
60260


### Preprocessing

In [6]:
# for decoder sequence
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary.head()

0            <go> 화성시 도로서 경차-트럭-트랙터 부딪혀…“2명 부상” <stop>
1                           <go> '홍대거리 방역조치 점검' <stop>
2    <go> 군산 만경강 야생조류 분변서 AI 항원 검출…"고병원성 여부 확인 중" <...
3         <go> 검찰, '소윤' 윤대진 친형 '변호사법 위반' 참고인 조사 <stop>
4                <go> 핼러윈 맞은 이태원 밤은...'기대 속 긴장' <stop>
Name: title, dtype: object

#### Tokenizing the texts into integer tokens

In [7]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'

In [8]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 200000,oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 200000, filters=filters, oov_token=oov_token)

In [9]:
document_tokenizer.fit_on_texts(document)
summary_tokenizer.fit_on_texts(summary)

In [10]:
inputs = document_tokenizer.texts_to_sequences(document)
targets = summary_tokenizer.texts_to_sequences(summary)

In [11]:
summary_tokenizer.texts_to_sequences(["월요일 발생했다."])

[[2019, 1]]

In [12]:
summary_tokenizer.sequences_to_texts([[184, 22, 12, 71]])

['구축 검찰 확진자 대상']

In [13]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(917272, 108308)

#### Obtaining insights on lengths for defining maxlen

In [14]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [15]:
document_lengths.describe()

count    60260.000000
mean       981.406356
std       1170.912433
min         57.000000
25%        507.750000
50%        801.000000
75%       1165.000000
max      40129.000000
dtype: float64

In [16]:
summary_lengths.describe()

count    60260.000000
mean        41.899635
std          6.847516
min         16.000000
25%         38.000000
50%         42.000000
75%         46.000000
max        132.000000
dtype: float64

In [17]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 400
decoder_maxlen = 75

#### Padding/Truncating sequences for identical sequence lengths

In [18]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [19]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [20]:
print(inputs.shape)
print(targets.shape)

(60260, 400)
(60260, 75)


In [21]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [23]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [24]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [25]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [26]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [27]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [28]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

### Feed Forward Network

In [29]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [30]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [31]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [32]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [33]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


#### Finally, the Transformer

In [34]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [35]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 12

#### Adam optimizer with custom learning rate scheduling

In [36]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [37]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [38]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [39]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [40]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [41]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

#### Masks

In [42]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [43]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


#### Training steps

In [44]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [45]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 258 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 3.7894
Epoch 1 Batch 258 Loss 3.7831
Epoch 1 Batch 516 Loss 3.8761
Epoch 1 Batch 774 Loss 3.9980
Epoch 1 Loss 4.1024
Time taken for 1 epoch: 844.0596919059753 secs

Epoch 2 Batch 0 Loss 3.7634
Epoch 2 Batch 258 Loss 3.7773
Epoch 2 Batch 516 Loss 3.8684
Epoch 2 Batch 774 Loss 3.9731
Epoch 2 Loss 4.0697
Time taken for 1 epoch: 827.1838057041168 secs

Epoch 3 Batch 0 Loss 3.7160
Epoch 3 Batch 258 Loss 3.7345
Epoch 3 Batch 516 Loss 3.8450
Epoch 3 Batch 774 Loss 3.9472
Epoch 3 Loss 4.0406
Time taken for 1 epoch: 822.4597451686859 secs

Epoch 4 Batch 0 Loss 4.0017
Epoch 4 Batch 258 Loss 3.7006
Epoch 4 Batch 516 Loss 3.7906
Epoch 4 Batch 774 Loss 3.9193
Epoch 4 Loss 4.0232
Time taken for 1 epoch: 824.7425858974457 secs

Epoch 5 Batch 0 Loss 3.7015
Epoch 5 Batch 258 Loss 3.6761
Epoch 5 Batch 516 Loss 3.7719
Epoch 5 Batch 774 Loss 3.8896
Saving checkpoint for epoch 5 at checkpoints/ckpt-9
Epoch 5 Loss 3.9858
Time taken for 1 epoch: 825.928300857544 secs

Epoch 6 Batch 0 Los

### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [46]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index["<go>"]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index["<stop>"]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [47]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003141547

In [48]:
summarize(
    '대한감염학회 등 전문가 단체가 국내 코로나19 유행 상황에 대해 거리두기 단계 상향 등 강력한 방역 조치가 없으면 향후 1~2주 안에 하루 1000명에 육박하는 신규 확진자가 발생할 것이라고 경고했다.\
    대한감염학회 등은 20일 성명서를 통해 “현재 코로나19 상황은 더 악화할 가능성이 높다”며 이같이 밝혔다.\
    이들 학회는 “코로나19 바이러스는 낮은 온도, 건조한 환경에서 더 오래 생존하므로 현재 전파 위험이 높아진 상태”라며 “일일 감염재생산 지수가 1.5를 넘어선 상태여서 효과적 조치 없이 1∼2주 경과하면 일일 확진자 수가 1000명에 육박할 것으로 예측된다”고 밝혔다.\
    이들 학회는 “고위험군에 피해가 발생할 위험이 커지고 있고, 코로나19 중환자를 치료할 자원이 빠르게 고갈되고 있다”며 “발병 후 7∼10일쯤 중증으로 악화하는 코로나19 특성을 고려하면 중환자 병상은 1∼2주 내 빠르게 소진될 것”이라고 진단했다. 이에 따라 조기에 선제적으로 강력하게 방역 조치를 해야 한다고 주문했다.')

'코로나19 피해 사상 최대 규모 조직 구성'

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003054205

In [49]:
summarize('서울 양천구의 한 인도에서 지나가던 행인을 무차별 폭행한 40대 남성 A씨가 20일 경찰에 붙잡혔다.\
서울 양천경찰서에 따르면, A씨는 지난 18일 오후 9시쯤 지하철 5호선 오목교역 근처에서 길을 가던 최모(42)씨와 어깨를 부딪치자 주먹을 휘두르고 발길질을 하며 수차례 폭행한 혐의를 받고 있다.\
신고를 받은 경찰이 출동했을 땐 A씨는 이미 현장에서 달아난 상태였다. 최씨는 당시 폭행으로 코뼈가 부러지고 뇌출혈 증상이 나타난 것으로 확인됐다.\
경찰은 주변 CCTV 등을 통해 A씨의 신원을 특정해 이틀 만에 덜미를 잡았다. 경찰 관계자는 “당시 A씨가 술에 취했던 것으로 추정된다”며 상해 등 혐의로 A씨를 입건했다.\
곧 A씨를 소환해 구체적인 범행 경위를 파악할 예정”이라고 말했다.'
         )

'마스크 착용 안 해 승차 정비 1심서 폭행'

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012030367

In [51]:
summarize("내 신종 코로나바이러스 감염증(코로나19) 확산세가 연일 거세지면서 21일 신규 확진자 수는 300명대 후반을 기록했다.\
전날(363명)보다 다소 늘어나면서 나흘 연속 300명대를 이어갔다.\
이는 수도권 중심의 '2차 유행'이 한창이던 8월 말 수준과 비슷한 상황이다. 당시엔 2차 유행의 정점을 찍었던 8월 27일(441명)을 전후로 4일 연속(320명→441명→371명→323명) 300명 이상이 단 1차례 있었다.\
그만큼 코로나19 상황이 심각하다는 방증으로, 정부도 지난 2∼3월 대구·경북 중심의 '1차 대유행'과 8월 2차 유행에 이어 '3차 유행'이 진행 중이라고 공식 확인한 상태다.\
이 같은 증가세는 기존 감염 사례에서 매일같이 확진자가 나오는 데다 학교나 학원, 종교시설, 각종 소모임 등을 고리로 전국 곳곳에서 크고 작은 집단발병이 연일 새로 발생하는 데 따른 것이다.\
정부는 환자 발생 동향을 주시하면서 수도권 등에 대한 '사회적 거리두기' 2단계 격상까지 열어두고 다각도의 대책을 모색하고 있다.")

'코로나19 중앙방역대책본부 브리핑'

In [52]:
summarize("중앙방역대책본부는 이날 0시 기준으로 코로나19 신규 확진자가 386명 늘어 누적3만403명이라고 밝혔다.\
전날(363명)과 비교하면 23명 늘었다.\
신규 확진자 386명은 8월 27일(441명) 이후 86일 만에 최다 기록이다.\
이달 들어 일별 신규 확진자 수는 124명→97명→75명→118명→125명→145명→89명→143명→126명→100명→146명→143명→191명→205명→208명→222명→230명→313명→343명→363명→386명 등이다. 지난 8일부터 2주 연속 세 자릿수를 이어간 가운데 300명대만 4차례다.\
이날 신규 확진자 386명의 감염경로를 보면 지역발생이 361명, 해외유입이 25명이다.\
지역발생 확진자는 지난 11일(113명) 이후 11일 연속 세 자릿수를 기록했으며, 수치상으로는 2차 유행의 정점이었던 8월 27일(434명) 이후 가장 많다.\
확진자가 나온 지역을 보면 서울 154명, 경기 86명, 인천 22명 등 수도권이 262명이다. 전날(218명)보다 44명 늘었다. 수도권 확진자가 연이틀 200명대를 기록한 것도 8월 29∼30일(244명→203명) 이후 처음이다.\
수도권 외 지역은 충남 19명, 전남 18명, 강원 14명, 전북 13명, 경남 11명, 경북 8명, 부산 7명, 광주 6명, 대전·울산·충북 각 1명이다. 비수도권 지역발생 확진자는 전날(102명)보다 3명 줄어든 99명으로, 100명에 육박했다.\
주요 감염 사례를 보면 수도권의 경우 전날 낮 12시까지 서울 동작구 노량진의 대형 교원 임용고시학원(누적 32명), 서대문구 연세대학교 학생모임(19명), 동대문구 고등학교(9명), 도봉구 종교시설 '청련사'(29명), 경기 안산시 수영장(17명), 인천 남동구 가족 및 지인(40명) 사례를 중심으로 확진자가 대거 나왔다.\
비수도권 지역에서는 충남 아산시 선문대학교(14명), 경남 창원시 친목모임(23명), 경남 하동군 중학교(26명), 전북 익산시 원광대병원(11명), 강원 철원군 장애인 요양원(40명), 광주 전남대병원(46명) 등 다양한 감염 고리를 통해 확진자가 잇따랐다.")

'코로나19 어제 확진 사흘째 세자리 지역 93명 해외 더 낮아'